In [13]:
import pandas as pd
from src.data import gedi_pipeline
from src.constants import DATA_PATH, USER_PATH
import seaborn as sns
import geopandas as gpd

from fastbook import *
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG
from sklearn.metrics import *

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8
import ee
import geopandas as gpd
from src.data import gedi_pipeline
from src.data.ee import lcms_import
from src.processing.regen import regen_analysis
from src.data import ee_utils
from src.data.fire_perimeters import FirePerimeters, FirePerimetersDB, Fire, match_gedi_to_raster, FireRastersDB
from src.data import fire_perimeters
from src.data import gedi_raster_matching
from src.data import raster
from src.processing.control import random_control
import rasterio as rio
import matplotlib.pyplot as plt
import pandas as pd
from src.constants import DATA_PATH, USER_PATH
from fitter import Fitter, get_common_distributions, get_distributions
import seaborn as sns
from src.processing.control import random_control, eval, nearest, terrain_clustering
from src.data import k_nn
from src.data import clustering
import numpy as np
from src.data import shape_processor
from src.processing.rf import split_data

import importlib
importlib.reload(regen_analysis)
importlib.reload(gedi_pipeline)
importlib.reload(ee_utils)
importlib.reload(gedi_raster_matching)
importlib.reload(raster)
importlib.reload(fire_perimeters)
importlib.reload(random_control)
importlib.reload(eval)
importlib.reload(terrain_clustering)
importlib.reload(k_nn)
importlib.reload(clustering)
importlib.reload(raster)

2023-06-13 17:32:09,081 DEBUG: Logger /home/jk871/fire-regen/src/data/gedi_pipeline.py already set up. [in get_logger at /home/jk871/fire-regen/src/utils/logging_util.py:51]
2023-06-13 17:32:09,082 DEBUG: Logger /home/jk871/fire-regen/src/data/ee_utils.py already set up. [in get_logger at /home/jk871/fire-regen/src/utils/logging_util.py:51]
2023-06-13 17:32:09,084 DEBUG: Logger /home/jk871/fire-regen/src/data/gedi_raster_matching.py already set up. [in get_logger at /home/jk871/fire-regen/src/utils/logging_util.py:51]
2023-06-13 17:32:09,086 DEBUG: Logger /home/jk871/fire-regen/src/data/fire_perimeters.py already set up. [in get_logger at /home/jk871/fire-regen/src/utils/logging_util.py:51]


<module 'src.data.raster' from '/home/jk871/fire-regen/src/data/raster.py'>

In [2]:
sierras = gpd.read_file(f"{USER_PATH}/data/shapefiles/sierras.shp")
sierras_box = shape_processor.get_box(sierras)

In [3]:
def r_mse(pred,y): return round(math.sqrt(mean_squared_error(y, pred)), 6)
def m_rmse(m, xs, y): return r_mse(m.predict(xs), y)
def r_squared(pred, y):
    return r2_score(y, pred)
def m_r2(m, xs, y): return r_squared(m.predict(xs), y)

In [4]:
sierras = gpd.read_file(f"{USER_PATH}/data/shapefiles/sierras_convex_hull.shp")
seki = gpd.read_file(f"{USER_PATH}/data/shapefiles/seki_convex_hull.shp")

In [5]:
gedi_burned, gedi_unburned = gedi_pipeline.load_stage_9(3)

In [9]:
spectral = list(gedi_raster_matching.LANDSAT8_ADV_BANDS.values())

In [18]:
spectral = list(gedi_raster_matching.LANDSAT8_ADV_BANDS.values())

def create_4_year_stack_of_landsat8_data(gedi: gpd.GeoDataFrame, year):
    rasters = []
    gedi_match = gedi.copy()

    for year in range(year, year + 4):
        print(f'Matching gedi shots from {year}')
        raster = gedi_raster_matching.get_landsat_raster_sampler(year)

        print(f'Sample raster')
        gedi_match = raster.sample_3x3(gedi_match, 'longitude', 'latitude')

        print(f'Process columns')
        gedi_match.rename(columns=dict(zip([f"{x}_mean" for x in spectral], [f"{x}_{year}" for x in spectral])), inplace=True)
        gedi_match = gedi_match.drop(columns=[f"{x}_3x3" for x in spectral] + [f"{x}_median" for x in spectral] + [f"{x}_std" for x in spectral])

    return gedi_match


In [19]:
gedi_matched = create_4_year_stack_of_landsat8_data(gedi_unburned, 2013)

Matching gedi shots from 2013
Sample raster
Process columns
Matching gedi shots from 2014
Sample raster
Process columns
Matching gedi shots from 2015
Sample raster
Process columns
Matching gedi shots from 2016
Sample raster


/maps-priv/maps/fire-regen/fire-regen-env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
/maps-priv/maps/fire-regen/fire-regen-env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
/maps-priv/maps/fire-regen/fire-regen-env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perf

Process columns


In [ ]:
from fastai import save_pickle

In [21]:
gedi_matched

,shot_number,longitude,latitude,agbd,...,SVVI_2016,brightness_2016,greenness_2016,wetness_2016
68,184891100200081436,-117.993053,37.035598,42.480953,...,0.0,0.317389,0.044610,-0.079194
69,184891100200081437,-117.992563,37.035940,41.270653,...,0.0,0.326052,0.042205,-0.081095
70,184891100200081438,-117.992074,37.036281,17.788033,...,0.0,0.322470,0.045124,-0.074491
71,184891100200081439,-117.991585,37.036622,33.051270,...,0.0,0.331342,0.043995,-0.082600
73,184891100200081441,-117.990609,37.037302,28.863516,...,0.0,0.316270,0.038827,-0.081900
...,...,...,...,...,...,...,...,...,...
27623080,42030300300286920,-119.951223,36.868402,15.163097,...,0.0,0.306733,0.149824,-0.016079
27623083,42030300300286932,-119.945451,36.864333,15.163097,...,0.0,0.314836,0.160930,-0.014573
27624716,187680600300533641,-119.909263,36.864995,19.482615,...,0.0,0.342519,0.153760,-0.022117
27626129,21830200200228428,-119.881873,36.860581,1.564893,...,0.0,0.372792,0.166644,-0.017677


In [73]:
spectral = [ "NDVI"]
years = [2013, 2014, 2015, 2016]
spectral_columns = [f"{x}_{y}" for x in spectral for y in years]
columns_to_use = ['agbd', 'pft_class', 'elevation', 'slope', 'aspect', 'soil', 'gedi_year']

In [52]:
gedi_unburned_ready = split_data.spatial_split_train_and_test_data(gedi_matched, sierras_box, 5000)

In [74]:
df = gedi_unburned_ready[columns_to_use]

In [75]:
procs = [Categorify, FillMissing, Normalize]
#procs = [Categorify, Normalize]

In [76]:
dep_var = 'agbd'

In [77]:
cont,cat = cont_cat_split(df, 1, dep_var=dep_var)


In [78]:
splits = IndexSplitter(gedi_unburned_ready[gedi_unburned_ready.dataset == "test"].index)((range_of(df)))

In [79]:
to = TabularPandas(df, procs, cat, cont, y_names=dep_var, splits=splits)

In [80]:
len(to.train),len(to.valid)


(1707744, 304218)

In [81]:
xs,y = to.train.xs,to.train.y
valid_xs,valid_y = to.valid.xs,to.valid.y

In [82]:
def rf(xs, y, n_estimators=100, max_samples=200_000,
       max_features=0.5, min_samples_leaf=5, **kwargs):
    return RandomForestRegressor(n_jobs=-1, n_estimators=n_estimators,
        max_samples=max_samples, max_features=max_features,
        min_samples_leaf=min_samples_leaf, oob_score=True).fit(xs, y)

In [83]:
m = rf(xs, y)

In [51]:
m_rmse(m, xs, y), m_rmse(m, valid_xs, valid_y), m_r2(m, xs, y), m_r2(m, valid_xs, valid_y)

(106.204806, 112.400351, 0.3812623280899755, 0.3040000904538761)

In [84]:
m_rmse(m, xs, y), m_rmse(m, valid_xs, valid_y), m_r2(m, xs, y), m_r2(m, valid_xs, valid_y)

(117.129873, 123.636398, 0.24741844892518816, 0.15789443984271445)